In [189]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline  # Use the imblearn pipeline

from sklearn.utils.validation import column_or_1d
from sklearn.ensemble import RandomForestClassifier

In [190]:
%run ./ml_model_init.ipynb

In [312]:
data_tar_preds = data_tar.copy()
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,npg_5,npxG_5,xGChain_5,xGBuildup_5,xP_5,selected_5,pts_bps,whh,whd,wha
11,0,2,33,1,0.0,14,0.00,0.00,0.00,1.56,...,0.0,0.00,0.45,0.45,7.1,1110544.0,7.0,0.38,0.29,0.38
21,0,0,3,0,4.1,388,0.03,0.03,0.00,2.17,...,0.0,0.11,0.84,0.67,7.3,911258.0,1.0,0.47,0.29,0.29
38,0,1,32,1,21.1,453,0.07,0.36,0.29,1.56,...,0.0,0.09,0.11,0.00,-1.5,247265.0,6.0,0.38,0.29,0.38
45,0,0,4,0,7.1,463,0.01,0.01,0.00,0.19,...,0.0,0.08,1.07,0.55,1.9,9944.0,1.0,0.38,0.29,0.38
56,0,0,7,0,15.4,81,0.01,0.61,0.60,1.56,...,0.0,0.00,0.26,0.23,9.9,107732.0,2.0,0.18,0.23,0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2285,0,0,12,0,1.3,291,0.01,0.12,0.11,1.00,...,0.0,0.48,1.08,1.06,9.0,6997283.0,2.0,0.44,0.28,0.32
2294,0,0,23,0,25.3,600,0.21,0.23,0.02,1.33,...,0.0,0.13,0.28,0.17,3.6,8560.0,2.0,0.71,0.20,0.13
2302,0,0,7,1,4.8,110,0.01,0.01,0.00,0.75,...,1.0,1.08,1.30,0.22,3.8,501058.0,2.0,0.49,0.27,0.29
2313,0,0,17,0,35.2,57,0.07,0.07,0.00,2.29,...,0.0,0.28,4.34,3.49,16.8,767011.0,2.0,0.52,0.25,0.27


## Functions


In [192]:
# For the linear model
def Linear_regression(features_train, features_pred, target_train):
    # bool_cols = features_train.drop(columns=['was_home']).columns.tolist()

    bool_cols = features_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = features_train.select_dtypes(include=['object', 'category']).columns.tolist()

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', LinearRegression())
    ])

    model = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())
    model.fit(features_train, target_train)

    return model.predict(features_pred)

# Decision Tree Model
def DecisionTreeRegression(features_train, features_pred, target_train):
    bool_cols = features_train.drop(columns=['was_home']).columns.tolist()
    categorical_cols = ['was_home']

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', DecisionTreeRegressor())
    ])

    model = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())
    model.fit(features_train, target_train)

    return model.predict(features_pred)

# RandomForestRegressor
def RandomForestRegression(features_train, features_pred, target_train, hyperparameters):
    model = TransformedTargetRegressor(RandomForestRegressor(
        n_estimators=hyperparameters['n_estimators'],  max_depth=hyperparameters['max_depth'], criterion=hyperparameters['criterion'], random_state=18), transformer=StandardScaler())
    model.fit(features_train, target_train)

    pred_pred = model.predict(features_pred)

def XGBoostRegression(features_train, features_pred, target_train, hyperparameters):
    regressor = xgb(learning_rate=hyperparameters["learning_rate"],
                    n_estimators=hyperparameters["n_estimators"],
                    max_depth=hyperparameters["max_depth"],
                    eval_metric='rmsle')

    model = TransformedTargetRegressor(regressor, transformer=StandardScaler())


    model.fit(features_train, target_train)

    return model.predict(features_pred)


def Random_Forest_Classifier(features_train, features_pred, target_train, hy_params):
    encoder = LabelEncoder()
    cs_train_ = encoder.fit_transform(target_train)

    # bool_cols = features_train.drop(columns=['was_home']).columns.tolist()
    # categorical_cols = ['was_home']
    bool_cols = features_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = features_train.select_dtypes(include=['object', 'category']).columns.tolist()

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder(sparse=False)),
        # ('to_dense', ToDense())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', BorderlineSMOTE(sampling_strategy='auto', random_state=42)),  # Apply SMOTE to the data
        ('classifier', RandomForestClassifier(bootstrap = hy_params['bootstrap'], min_samples_split = hy_params['min_samples_split'], n_estimators =hy_params['n_estimators']  , class_weight='balanced', random_state=42))  # Random Forest Classifier
    ])
# {}
    pipeline.fit(features_train, cs_train_)

    # Make predictions
    return pipeline.predict(features_pred)

## Goals Predictor


In [333]:
xg_data= data[(data['minutes_5'] >= 300) & (data['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3',  'ict_index_5',
                                                                     'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5', 'was_home', 'xG', 'xG_3',
                                                                      'xG_5', 'expected_goals_3', 'expected_goals_5', 'goals_scored_3', 'whh', 'whd', 'wha']]

xg_data_tar = data_tar[(data_tar['minutes_5'] >= 300) & (data_tar['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3',
                                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3',
                                                                                        'threat_5', 'was_home', 'xG', 'xG_3', 'xG_5', 'expected_goals_3', 'expected_goals_5',
                                                                                        'goals_scored_3','whh', 'whd', 'wha']]

In [349]:

xg = xg_data[['xG']]
feats = xg_data.drop('xG', axis=1)

xg_tar = xg_data_tar[['xG']].copy()
feats_tar = xg_data_tar.drop('xG', axis=1)

In [350]:
xg_pred = Linear_regression(feats, feats_tar, xg)
xg_tar['xg_pred'] = xg_pred
# xg_tar = xg_tar.drop('xG', axis=1)
# comb_data = xg_data_tar.join(xg_tar)
# comb_data

def add_preds(row):
    if row.name in list(xg_tar.index):
        # print(row.name, xg_tar.loc[row.name, 'xg_pred'])
        row['xg_pred'] = xg_tar.loc[row.name, 'xg_pred']
    else:
        row['xg_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,selected_5,pts_bps,whh,whd,wha,xg_pred,xa_pred,cs_pred,sv_pred,yc_pred
11,0,2,33,1,0.0,14,0.00,0.00,0.00,1.56,...,1110544.0,7.0,0.38,0.29,0.38,0.000000,NaN,0.0,3.302557,NaN
21,0,0,3,0,4.1,388,0.03,0.03,0.00,2.17,...,911258.0,1.0,0.47,0.29,0.29,0.056395,0.095109,0.0,NaN,0.0
38,0,1,32,1,21.1,453,0.07,0.36,0.29,1.56,...,247265.0,6.0,0.38,0.29,0.38,0.000000,NaN,NaN,NaN,NaN
45,0,0,4,0,7.1,463,0.01,0.01,0.00,0.19,...,9944.0,1.0,0.38,0.29,0.38,0.000000,NaN,NaN,NaN,NaN
56,0,0,7,0,15.4,81,0.01,0.61,0.60,1.56,...,107732.0,2.0,0.18,0.23,0.62,0.000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2285,0,0,12,0,1.3,291,0.01,0.12,0.11,1.00,...,6997283.0,2.0,0.44,0.28,0.32,0.041014,0.031819,0.0,NaN,1.0
2294,0,0,23,0,25.3,600,0.21,0.23,0.02,1.33,...,8560.0,2.0,0.71,0.20,0.13,0.000000,NaN,NaN,NaN,NaN
2302,0,0,7,1,4.8,110,0.01,0.01,0.00,0.75,...,501058.0,2.0,0.49,0.27,0.29,0.000000,NaN,NaN,NaN,NaN
2313,0,0,17,0,35.2,57,0.07,0.07,0.00,2.29,...,767011.0,2.0,0.52,0.25,0.27,0.117886,0.167997,0.0,NaN,0.0


## Assists Predictor


In [336]:

xa_data= data[(data['minutes_5'] >= 300) & (data['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5', 'was_home', 'expected_assists', 'xA_3', 'xA_5',  'expected_assists_3', 'expected_assists_5', 'whh', 'whd', 'wha']]

xa_data_tar = data_tar[(data_tar['minutes_5'] >= 300) & (data_tar['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5','was_home', 'expected_assists', 'xA_3', 'xA_5',  'expected_assists_3',
                    'expected_assists_5', 'whh', 'whd', 'wha']]



In [353]:

xa = xa_data[['expected_assists']]
feats = xa_data.drop('expected_assists', axis=1)

xa_tar = xa_data_tar[['expected_assists']].copy()
feats_tar = xa_data_tar.drop('expected_assists', axis=1)

In [354]:
xa_pred = Linear_regression(feats, feats_tar, xa)
xa_tar['xa_pred'] = xa_pred
# xa_tar = xa_tar.drop('expected_assists', axis=1)
# comb_data = comb_data.join(xa_tar, rsuffix='x')
# comb_data

def add_preds(row):
    if row.name in list(xa_tar.index):
        # print(row.name, xa_tar.loc[row.name, 'xa_pred'])
        row['xa_pred'] = xa_tar.loc[row.name, 'xa_pred']
    else:
        row['xa_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,selected_5,pts_bps,whh,whd,wha,xg_pred,xa_pred,cs_pred,sv_pred,yc_pred
11,0,2,33,1,0.0,14,0.00,0.00,0.00,1.56,...,1110544.0,7.0,0.38,0.29,0.38,0.000000,0.000000,0.0,3.302557,NaN
21,0,0,3,0,4.1,388,0.03,0.03,0.00,2.17,...,911258.0,1.0,0.47,0.29,0.29,0.056395,0.095109,0.0,NaN,0.0
38,0,1,32,1,21.1,453,0.07,0.36,0.29,1.56,...,247265.0,6.0,0.38,0.29,0.38,0.000000,0.000000,NaN,NaN,NaN
45,0,0,4,0,7.1,463,0.01,0.01,0.00,0.19,...,9944.0,1.0,0.38,0.29,0.38,0.000000,0.000000,NaN,NaN,NaN
56,0,0,7,0,15.4,81,0.01,0.61,0.60,1.56,...,107732.0,2.0,0.18,0.23,0.62,0.000000,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2285,0,0,12,0,1.3,291,0.01,0.12,0.11,1.00,...,6997283.0,2.0,0.44,0.28,0.32,0.041014,0.031819,0.0,NaN,1.0
2294,0,0,23,0,25.3,600,0.21,0.23,0.02,1.33,...,8560.0,2.0,0.71,0.20,0.13,0.000000,0.000000,NaN,NaN,NaN
2302,0,0,7,1,4.8,110,0.01,0.01,0.00,0.75,...,501058.0,2.0,0.49,0.27,0.29,0.000000,0.000000,NaN,NaN,NaN
2313,0,0,17,0,35.2,57,0.07,0.07,0.00,2.29,...,767011.0,2.0,0.52,0.25,0.27,0.117886,0.167997,0.0,NaN,0.0


## Clean Sheets


In [339]:
cs_data= data[(data['minutes_5'] >= 300) & (data['position'] != 'FWD')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'clean_sheets', 'clean_sheets_3', 'clean_sheets_5',
                                                                         'expected_goals_conceded_3', 'expected_goals_conceded_5', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5', 'whh', 'whd', 'wha']]

cs_data_tar = data_tar[(data_tar['minutes_5'] >= 300) & (data_tar['position'] != 'FWD')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'clean_sheets', 'clean_sheets_3', 'clean_sheets_5',
                                                                         'expected_goals_conceded_3', 'expected_goals_conceded_5', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5', 'whh', 'whd', 'wha']]

In [355]:
cs = cs_data[['clean_sheets']]
feats = cs_data.drop('clean_sheets', axis=1)

cs_tar = cs_data_tar[['clean_sheets']].copy()
feats_tar = cs_data_tar.drop('clean_sheets', axis=1)

In [356]:
cs_hy_params = {'bootstrap': False, 'min_samples_split': 2, 'n_estimators': 100}
cs_pred = Random_Forest_Classifier(feats, feats_tar, column_or_1d(cs), cs_hy_params)
cs_tar['cs_pred'] = cs_pred
# cs_tar = cs_tar.drop('clean_sheets', axis=1)
# comb_data = comb_data.join(cs_tar, rsuffix='x')
# comb_data

def add_preds(row):
    if row.name in list(cs_tar.index):
        # print(row.name, cs_tar.loc[row.name, 'cs_pred'])
        row['cs_pred'] = cs_tar.loc[row.name, 'cs_pred']
    else:
        row['cs_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,selected_5,pts_bps,whh,whd,wha,xg_pred,xa_pred,cs_pred,sv_pred,yc_pred
11,0,2,33,1,0.0,14,0.00,0.00,0.00,1.56,...,1110544.0,7.0,0.38,0.29,0.38,0.000000,0.000000,0,3.302557,NaN
21,0,0,3,0,4.1,388,0.03,0.03,0.00,2.17,...,911258.0,1.0,0.47,0.29,0.29,0.056395,0.095109,0,NaN,0.0
38,0,1,32,1,21.1,453,0.07,0.36,0.29,1.56,...,247265.0,6.0,0.38,0.29,0.38,0.000000,0.000000,0,NaN,NaN
45,0,0,4,0,7.1,463,0.01,0.01,0.00,0.19,...,9944.0,1.0,0.38,0.29,0.38,0.000000,0.000000,0,NaN,NaN
56,0,0,7,0,15.4,81,0.01,0.61,0.60,1.56,...,107732.0,2.0,0.18,0.23,0.62,0.000000,0.000000,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2285,0,0,12,0,1.3,291,0.01,0.12,0.11,1.00,...,6997283.0,2.0,0.44,0.28,0.32,0.041014,0.031819,0,NaN,1.0
2294,0,0,23,0,25.3,600,0.21,0.23,0.02,1.33,...,8560.0,2.0,0.71,0.20,0.13,0.000000,0.000000,0,NaN,NaN
2302,0,0,7,1,4.8,110,0.01,0.01,0.00,0.75,...,501058.0,2.0,0.49,0.27,0.29,0.000000,0.000000,0,NaN,NaN
2313,0,0,17,0,35.2,57,0.07,0.07,0.00,2.29,...,767011.0,2.0,0.52,0.25,0.27,0.117886,0.167997,0,NaN,0.0


## Yellow Cards


In [342]:
#
yc_data= data[(data['minutes_5'] >= 300) & (data['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'yellow_cards', 'yellow_cards_3', 'yellow_cards_5', 'whh', 'whd', 'wha']]

yc_data_tar = data_tar[(data_tar['minutes_5'] >= 300) & (data_tar['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'yellow_cards', 'yellow_cards_3', 'yellow_cards_5', 'whh', 'whd', 'wha']]


In [357]:
yc = yc_data[['yellow_cards']]
feats = yc_data.drop('yellow_cards', axis=1)

yc_tar = yc_data_tar[['yellow_cards']].copy()
feats_tar = yc_data_tar.drop('yellow_cards', axis=1)

In [358]:
params =  {'bootstrap': False, 'min_samples_split': 10, 'n_estimators': 300}
yc_pred = Random_Forest_Classifier(feats, feats_tar, column_or_1d(yc), params)
yc_tar['yc_pred'] = yc_pred
# yc_tar = yc_tar.drop('yellow_cards', axis=1)
# comb_data = comb_data.merge(yc_tar, left_index=True, right_index=True, how='inner')
# comb_data

def add_preds(row):
    if row.name in list(yc_tar.index):
        # print(row.name, yc_tar.loc[row.name, 'yc_pred'])
        row['yc_pred'] = yc_tar.loc[row.name, 'yc_pred']
    else:
        row['yc_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,selected_5,pts_bps,whh,whd,wha,xg_pred,xa_pred,cs_pred,sv_pred,yc_pred
11,0,2,33,1,0.0,14,0.00,0.00,0.00,1.56,...,1110544.0,7.0,0.38,0.29,0.38,0.000000,0.000000,0,3.302557,0
21,0,0,3,0,4.1,388,0.03,0.03,0.00,2.17,...,911258.0,1.0,0.47,0.29,0.29,0.056395,0.095109,0,NaN,0
38,0,1,32,1,21.1,453,0.07,0.36,0.29,1.56,...,247265.0,6.0,0.38,0.29,0.38,0.000000,0.000000,0,NaN,0
45,0,0,4,0,7.1,463,0.01,0.01,0.00,0.19,...,9944.0,1.0,0.38,0.29,0.38,0.000000,0.000000,0,NaN,0
56,0,0,7,0,15.4,81,0.01,0.61,0.60,1.56,...,107732.0,2.0,0.18,0.23,0.62,0.000000,0.000000,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2285,0,0,12,0,1.3,291,0.01,0.12,0.11,1.00,...,6997283.0,2.0,0.44,0.28,0.32,0.041014,0.031819,0,NaN,1
2294,0,0,23,0,25.3,600,0.21,0.23,0.02,1.33,...,8560.0,2.0,0.71,0.20,0.13,0.000000,0.000000,0,NaN,0
2302,0,0,7,1,4.8,110,0.01,0.01,0.00,0.75,...,501058.0,2.0,0.49,0.27,0.29,0.000000,0.000000,0,NaN,0
2313,0,0,17,0,35.2,57,0.07,0.07,0.00,2.29,...,767011.0,2.0,0.52,0.25,0.27,0.117886,0.167997,0,NaN,0


## Saves


In [ ]:
sv_data= data[(data['minutes_5'] >= 300) & (data['position'] == 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'clean_sheets_3', 'clean_sheets_5', 'expected_goals_conceded_3', 'expected_goals_conceded_5','saves',
                                                                          'saves_3', 'saves_5', 'whh', 'whd', 'wha']]

sv_data_tar = data_tar[(data_tar['minutes_5'] >= 300) & (data_tar['position'] == 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'clean_sheets_3', 'clean_sheets_5', 'expected_goals_conceded_3', 'expected_goals_conceded_5','saves',
                                                                          'saves_3', 'saves_5', 'whh', 'whd', 'wha']]


In [359]:
sv = sv_data[['saves']]
feats = sv_data.drop('saves', axis=1)

sv_tar = sv_data_tar[['saves']].copy()
feats_tar = sv_data_tar.drop('saves', axis=1)

In [360]:
sv_pred = Linear_regression(feats, feats_tar, sv)
sv_tar['sv_pred'] = sv_pred
# sv_tar = sv_tar.drop('saves', axis=1)
# comb_data = comb_data.join(sv_tar, rsuffix='x')
# comb_data

def add_preds(row):
    if row.name in list(sv_tar.index):
        # print(row.name, sv_tar.loc[row.name, 'sv_pred'])
        row['sv_pred'] = sv_tar.loc[row.name, 'sv_pred']
    else:
        row['sv_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,selected_5,pts_bps,whh,whd,wha,xg_pred,xa_pred,cs_pred,sv_pred,yc_pred
11,0,2,33,1,0.0,14,0.00,0.00,0.00,1.56,...,1110544.0,7.0,0.38,0.29,0.38,0.000000,0.000000,0,3.302557,0
21,0,0,3,0,4.1,388,0.03,0.03,0.00,2.17,...,911258.0,1.0,0.47,0.29,0.29,0.056395,0.095109,0,0.000000,0
38,0,1,32,1,21.1,453,0.07,0.36,0.29,1.56,...,247265.0,6.0,0.38,0.29,0.38,0.000000,0.000000,0,0.000000,0
45,0,0,4,0,7.1,463,0.01,0.01,0.00,0.19,...,9944.0,1.0,0.38,0.29,0.38,0.000000,0.000000,0,0.000000,0
56,0,0,7,0,15.4,81,0.01,0.61,0.60,1.56,...,107732.0,2.0,0.18,0.23,0.62,0.000000,0.000000,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2285,0,0,12,0,1.3,291,0.01,0.12,0.11,1.00,...,6997283.0,2.0,0.44,0.28,0.32,0.041014,0.031819,0,0.000000,1
2294,0,0,23,0,25.3,600,0.21,0.23,0.02,1.33,...,8560.0,2.0,0.71,0.20,0.13,0.000000,0.000000,0,0.000000,0
2302,0,0,7,1,4.8,110,0.01,0.01,0.00,0.75,...,501058.0,2.0,0.49,0.27,0.29,0.000000,0.000000,0,0.000000,0
2313,0,0,17,0,35.2,57,0.07,0.07,0.00,2.29,...,767011.0,2.0,0.52,0.25,0.27,0.117886,0.167997,0,0.000000,0
